# Baseline Modular Version

**목적**: 모듈화된 코드를 사용하여 baseline을 재현합니다.

**기대 결과**: baseline.ipynb와 동일한 성능 (46-47점)

---

## 1. Import 모듈

In [ ]:
import sys
import os

# scripts 디렉토리를 Python path에 추가
sys.path.append('../scripts')

import torch
from torch.utils.data import DataLoader

# 모듈 import
from utils import load_config, get_device, set_seed
from data_loader import Preprocess, load_data
from tokenizer_utils import load_tokenizer
from model_utils import load_model_for_train, get_model_info
from dataset import prepare_train_dataset, prepare_test_dataset
from trainer_utils import get_trainer
from inference_utils import run_inference

print("✅ 모든 모듈 import 완료")

## 2. Config 로드 및 환경 설정

In [ ]:
# Config 로드
config = load_config('./config.yaml')

# Device 설정
device = get_device()
print(f"디바이스: {device}")

# 시드 설정 (재현성)
set_seed(config['training']['seed'])
print(f"시드 설정 완료: {config['training']['seed']}")

## 3. Wandb 설정 (선택적)

In [ ]:
# Wandb 사용 시 주석 해제
# import wandb
# from dotenv import load_dotenv
# 
# load_dotenv()
# wandb.login()
# wandb.init(
#     project=config['wandb']['project'],
#     entity=config['wandb']['entity'],
#     name=config['wandb']['name'] + "-modular"
# )

# Wandb 비활성화 (기본값)
config['training']['report_to'] = 'none'
print("Wandb 비활성화 (report_to='none')")

## 4. Tokenizer 로드

In [ ]:
# Tokenizer 로드 (special tokens 추가)
model_name = config['general']['model_name']
special_tokens = config['tokenizer']['special_tokens']

tokenizer = load_tokenizer(model_name, special_tokens)

print(f"✅ Tokenizer 로드 완료")
print(f"모델: {model_name}")
print(f"Vocab size: {len(tokenizer)}")
print(f"Special tokens: {special_tokens}")

## 5. 데이터셋 준비

In [ ]:
# Preprocessor 생성
preprocessor = Preprocess(
    bos_token=config['tokenizer']['bos_token'],
    eos_token=config['tokenizer']['eos_token']
)

# Train/Val 데이터셋 준비
data_path = config['general']['data_path']
train_dataset, val_dataset = prepare_train_dataset(
    config, preprocessor, data_path, tokenizer
)

print(f"✅ 데이터셋 준비 완료")
print(f"Train: {len(train_dataset)} samples")
print(f"Val: {len(val_dataset)} samples")

# 샘플 확인
print("\n샘플 데이터:")
sample = train_dataset[0]
for key, value in sample.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}")
    else:
        print(f"  {key}: {value}")

## 6. 모델 로드

In [ ]:
# 모델 로드
model = load_model_for_train(config, tokenizer, device)

print(f"✅ 모델 로드 완료")

# 모델 정보 출력
model_info = get_model_info(model)
print("\n모델 정보:")
for key, value in model_info.items():
    if 'parameters' in key:
        print(f"  {key}: {value:,}")
    else:
        print(f"  {key}: {value}")

## 7. Trainer 설정 및 학습

In [ ]:
# Trainer 생성
trainer = get_trainer(
    config=config,
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    val_dataset=val_dataset
)

print("✅ Trainer 설정 완료")
print(f"학습 에폭: {config['training']['num_train_epochs']}")
print(f"학습률: {config['training']['learning_rate']}")
print(f"배치 크기: {config['training']['per_device_train_batch_size']}")

In [ ]:
# 학습 시작
print("\n🚀 학습 시작...\n")
trainer.train()
print("\n✅ 학습 완료!")

## 8. Test 데이터 추론

In [ ]:
# Test 데이터셋 준비
test_data, test_dataset = prepare_test_dataset(config, preprocessor, tokenizer)

print(f"✅ Test 데이터셋 준비 완료")
print(f"Test: {len(test_dataset)} samples")

# DataLoader 생성
test_dataloader = DataLoader(
    test_dataset,
    batch_size=config['inference']['batch_size']
)

In [ ]:
# 추론 실행
print("\n🔮 추론 시작...\n")

result_df = run_inference(
    model=model,
    tokenizer=tokenizer,
    test_dataloader=test_dataloader,
    config=config,
    device=device,
    save_path='./prediction/output_modular.csv'
)

print("\n✅ 추론 완료!")
print(f"결과 파일: ./prediction/output_modular.csv")
print(f"샘플 수: {len(result_df)}")

## 9. 결과 확인

In [ ]:
# 처음 5개 샘플 출력
print("\n샘플 결과 (처음 5개):")
print("=" * 80)
for i in range(min(5, len(result_df))):
    print(f"\n[{i}] {result_df.iloc[i]['fname']}")
    print(f"요약: {result_df.iloc[i]['summary'][:100]}...")
print("=" * 80)

In [ ]:
# CSV 검증
from utils import validate_csv

validation_result = validate_csv('./prediction/output_modular.csv')

print("\nCSV 검증 결과:")
print(f"유효성: {'✅ 통과' if validation_result['valid'] else '❌ 실패'}")
print(f"샘플 수: {validation_result['num_samples']}")
print(f"컬럼: {validation_result['columns']}")

if validation_result['errors']:
    print("\n⚠️ 오류:")
    for error in validation_result['errors']:
        print(f"  - {error}")

## 10. 원본 Baseline과 비교 (선택적)

In [ ]:
# 원본 baseline.ipynb의 결과와 비교
import pandas as pd

try:
    baseline_output = pd.read_csv('./prediction/output.csv')
    modular_output = pd.read_csv('./prediction/output_modular.csv')
    
    # 동일한 샘플 수인지 확인
    print(f"\nBaseline 샘플 수: {len(baseline_output)}")
    print(f"Modular 샘플 수: {len(modular_output)}")
    
    # fname 순서가 동일한지 확인
    if baseline_output['fname'].equals(modular_output['fname']):
        print("✅ fname 순서 일치")
    else:
        print("⚠️ fname 순서 불일치")
    
    # 일치하는 샘플 수 계산
    identical_count = (baseline_output['summary'] == modular_output['summary']).sum()
    print(f"\n동일한 요약문 수: {identical_count} / {len(baseline_output)}")
    print(f"일치율: {identical_count / len(baseline_output) * 100:.2f}%")
    
except FileNotFoundError:
    print("⚠️ 원본 baseline 결과 파일(output.csv)을 찾을 수 없습니다.")
    print("baseline.ipynb를 먼저 실행해주세요.")

---

## ✅ 완료

**다음 단계**:
1. `./prediction/output_modular.csv`를 대회 플랫폼에 제출
2. 점수 확인 (46-47점 기대)
3. 모듈화된 코드를 활용하여 실험 진행 (Learning rate 튜닝, 데이터 증강 등)

**모듈 사용 방법**:
- 새로운 실험을 위해 이 notebook을 복사하고 config를 수정
- 각 모듈은 독립적으로 수정 가능
- 모든 실험은 notebook으로 추적 가능